# Final Project
### Alex Ledgerwood

Github Repo: https://github.com/ALedgerwood/Module-7-Final-Project
Link to Tutorial:https://realpython.com/natural-language-processing-spacy-python/
Link to Text to be analyzed:https://readingwise.com/assets/uploads/pdf/The_Hill_We_Climb_Transcript.pdf

### import spacy and define nlp

In [15]:
import spacy
nlp = spacy.load("en_core_web_sm")
nlp


### call the .text attribute on each token to get the text contained within that token.

In [18]:
introduction_doc = nlp("This tutorial is about Natural Language Processing in spaCy.")
type(introduction_doc)

[token.text for token in introduction_doc]

['This',
 'tutorial',
 'is',
 'about',
 'Natural',
 'Language',
 'Processing',
 'in',
 'spaCy',
 '.']

### do the same thing but reading from a file, not typing it in

I don't know where the introduction.txt file is supposed to be - I am using the exact code from the tutorial

>>> import pathlib
>>> file_name = "introduction.txt"
>>> introduction_doc = nlp(pathlib.Path(file_name).read_text(encoding="utf-8"))
>>> print ([token.text for token in introduction_doc])

### the .sents property is used to extract sentences from the Doc object

In [22]:
>>> about_text = (
...     "Gus Proto is a Python developer currently"
...     " working for a London-based Fintech"
...     " company. He is interested in learning"
...     " Natural Language Processing."
... )
>>> about_doc = nlp(about_text)
>>> sentences = list(about_doc.sents)
>>> len(sentences)
2
>>> for sentence in sentences:
...     print(f"{sentence[:5]}...")

Gus Proto is a Python...
He is interested in learning...


### Creating custom delimeters in sentence detection

this uses elipses as a delimeter

In [23]:
>>> ellipsis_text = (
...     "Gus, can you, ... never mind, I forgot"
...     " what I was saying. So, do you think"
...     " we should ..."
... )

>>> from spacy.language import Language
>>> @Language.component("set_custom_boundaries")
... def set_custom_boundaries(doc):
...     """Add support to use `...` as a delimiter for sentence detection"""
...     for token in doc[:-1]:
...         if token.text == "...":
...             doc[token.i + 1].is_sent_start = True
...     return doc
...

>>> custom_nlp = spacy.load("en_core_web_sm")
>>> custom_nlp.add_pipe("set_custom_boundaries", before="parser")
>>> custom_ellipsis_doc = custom_nlp(ellipsis_text)
>>> custom_ellipsis_sentences = list(custom_ellipsis_doc.sents)
>>> for sentence in custom_ellipsis_sentences:
...     print(sentence)

Gus, can you, ...
never mind, I forgot what I was saying.
So, do you think we should ...


### Tokenization - showing that the token’s original index position in the string is still available as an attribute on Token

could be useful for in-place word replacement down the line

In [24]:
>>> import spacy
>>> nlp = spacy.load("en_core_web_sm")
>>> about_text = (
...     "Gus Proto is a Python developer currently"
...     " working for a London-based Fintech"
...     " company. He is interested in learning"
...     " Natural Language Processing."
... )
>>> about_doc = nlp(about_text)

>>> for token in about_doc:
...     print (token, token.idx)

Gus 0
Proto 4
is 10
a 13
Python 15
developer 22
currently 32
working 42
for 50
a 54
London 56
- 62
based 63
Fintech 69
company 77
. 84
He 86
is 89
interested 92
in 103
learning 106
Natural 115
Language 123
Processing 132
. 142


### Other attributes for the token class
use f-string formatting to output a table accessing some common attributes from each Token in Doc:

.text_with_ws prints the token text along with any trailing space, if present.

.is_alpha indicates whether the token consists of alphabetic characters or not.

.is_punct indicates whether the token is a punctuation symbol or not.

.is_stop indicates whether the token is a stop word or not. 

In [30]:
>>> print(
...     f"{'Text with Whitespace':22}"
...     f"{'Is Alphanumeric?':15}"
...     f"{'Is Punctuation?':18}"
...     f"{'Is Stop Word?'}"
... )
>>> for token in about_doc:
...     print(
...         f"{str(token.text_with_ws):22}"
...         f"{str(token.is_alpha):15}"
...         f"{str(token.is_punct):18}"
...         f"{str(token.is_stop)}"
...     )
...

Text with Whitespace  Is Alphanumeric?Is Punctuation?   Is Stop Word?
Gus                   True           False             False
Proto                 True           False             False
is                    True           False             True
a                     True           False             True
Python                True           False             False
developer             True           False             False
currently             True           False             False
working               True           False             False
for                   True           False             True
a                     True           False             True
London                True           False             False
-                     False          True              False
based                 True           False             False
Fintech               True           False             False
company               True           False             False
.                  

### custom tokenization

In [31]:
>>> custom_about_text = (
...     "Gus Proto is a Python developer currently"
...     " working for a London@based Fintech"
...     " company. He is interested in learning"
...     " Natural Language Processing."
... )

>>> print([token.text for token in nlp(custom_about_text)[8:15]])

['for', 'a', 'London@based', 'Fintech', 'company', '.', 'He']


### To include the @ symbol as a custom infix, you need to build your own Tokenizer object

In [32]:
>>> import re
>>> from spacy.tokenizer import Tokenizer

>>> custom_nlp = spacy.load("en_core_web_sm")
>>> prefix_re = spacy.util.compile_prefix_regex(
...     custom_nlp.Defaults.prefixes
... )
>>> suffix_re = spacy.util.compile_suffix_regex(
...     custom_nlp.Defaults.suffixes
... )

>>> custom_infixes = [r"@"]

>>> infix_re = spacy.util.compile_infix_regex(
...     list(custom_nlp.Defaults.infixes) + custom_infixes
... )

>>> custom_nlp.tokenizer = Tokenizer(
...     nlp.vocab,
...     prefix_search=prefix_re.search,
...     suffix_search=suffix_re.search,
...     infix_finditer=infix_re.finditer,
...     token_match=None,
... )

>>> custom_tokenizer_about_doc = custom_nlp(custom_about_text)

>>> print([token.text for token in custom_tokenizer_about_doc[8:15]])

['for', 'a', 'London', '@', 'based', 'Fintech', 'company']


### Stop Words
Stop words are typically defined as the most common words in a language.

With NLP, stop words are generally removed because they aren’t significant, and they heavily distort any word frequency analysis. 

In [33]:
>>> import spacy
>>> spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS
>>> len(spacy_stopwords)
326
>>> for stop_word in list(spacy_stopwords)[:10]:
...     print(stop_word)

him
seem
had
then
should
move
’m
becoming
yours
every


### .is_stop attribute
You don’t need to access this list directly, though. You can remove stop words from the input text by making use of the .is_stop attribute of each token:

In [34]:
>>> custom_about_text = (
...     "Gus Proto is a Python developer currently"
...     " working for a London-based Fintech"
...     " company. He is interested in learning"
...     " Natural Language Processing."
... )
>>> nlp = spacy.load("en_core_web_sm")
>>> about_doc = nlp(custom_about_text)
>>> print([token for token in about_doc if not token.is_stop])

[Gus, Proto, Python, developer, currently, working, London, -, based, Fintech, company, ., interested, learning, Natural, Language, Processing, .]
